In [284]:
import xarray as xr
import altair as alt

In [285]:
ds = xr.open_dataset("/Users/elischwat/Downloads/isfs_20221030.nc")

## Declare variable prefix and suffix to filter data

In [286]:
VARIABLE_PREFIX = 'spd'
TOWER_SUFFIX = 'c'

In [287]:
variables = [f for f in list(ds.data_vars) if VARIABLE_PREFIX in f and TOWER_SUFFIX in f]
variables


['spd_5m_c',
 'spd_20m_c',
 'spd_1m_c',
 'spd_3m_c',
 'spd_2m_c',
 'spd_15m_c',
 'spd_10m_c']

## Convert dataset to dataframe

In [288]:
src = ds[variables].to_dataframe()

## Parse height information from column names so measurement height is a column/index

In [265]:
def height_from_variable_name(name):
    if '_1m_' in name:
        return 1
    elif '_2m_' in name:
        return 2
    elif '_3m_' in name:
        return 3
    elif '_5m_' in name:
        return 5
    elif '_10m_' in name:
        return 10
    elif '_15m_' in name:
        return 15
    elif '_20m_' in name:
        return 20

In [289]:
src = src.reset_index()
src = src.melt(id_vars='time', value_vars=[col for col in src.columns if 'spd' in col])
src['height'] = src['variable'].apply(height_from_variable_name)
src = src.set_index(['time'])

In [290]:
src

,variable,value,height
time,,,
2022-10-30 00:02:30,spd_5m_c,4.680837,5
2022-10-30 00:07:30,spd_5m_c,4.051001,5
2022-10-30 00:12:30,spd_5m_c,4.099550,5
2022-10-30 00:17:30,spd_5m_c,4.465554,5
2022-10-30 00:22:30,spd_5m_c,4.979016,5
...,...,...,...
2022-10-30 23:37:30,spd_10m_c,3.929428,10
2022-10-30 23:42:30,spd_10m_c,4.502552,10
2022-10-30 23:47:30,spd_10m_c,5.680924,10


## Resample to 3 hour means

In [291]:
src = src.groupby(['height', 'variable']).resample('3H').mean()
src = src.drop(columns='height')

/var/folders/jy/xv6qnlms2cj_3p5yrzcwc0n00000gn/T/ipykernel_86078/216488169.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  src = src.groupby(['height', 'variable']).resample('3H').mean()


## Plot

In [292]:
alt.Chart(src.reset_index()).mark_line().encode(
    alt.X('value:Q', sort='y', ),
    alt.Y('height:Q', title='Height (m)'),
    alt.Color('time:O', scale=alt.Scale(scheme='inferno'))
)

/usr/local/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)